In [2]:
!pip install category_encoders

     |████████████████████████████████| 81kB 5.2MB/s 


In [3]:
import pandas as pd 
import os 
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from pickle import dump


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [59]:
df = pd.read_csv('/content/drive/MyDrive/IMDb movies.csv',encoding = "ISO-8859-1") #從google雲端拿取資料

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [60]:
df = df.drop(columns=['year','title','imdb_title_id','original_title','date_published','production_company','description','votes', 'budget','usa_gross_income','worlwide_gross_income','metascore','reviews_from_users','reviews_from_critics'])

In [61]:
df["genre2"] = df["genre2"].fillna("None")
df["genre3"] = df["genre3"].fillna("None")
df["actors2"] = df["actors2"].fillna("None")
df["actors3"] = df["actors3"].fillna("None")
df = df.dropna(how='any')  

In [62]:
df['genre'] = df.genre.str.strip()
df['genre2'] = df.genre2.str.strip()
df['genre3'] = df.genre3.str.strip()
df['country'] = df.country.str.strip()
df['language'] = df.language.str.strip()
df['director'] = df.director.str.strip()
df['writer'] = df.writer.str.strip()
df['actors'] = df.actors.str.strip()
df['actors2'] = df.actors2.str.strip()
df['actors3'] = df.actors3.str.strip()

df['genre'] = df.genre.str.replace('[^a-zA-Z ]', '')
df['genre2'] = df.genre2.str.replace('[^a-zA-Z ]', '')
df['genre3'] = df.genre3.str.replace('[^a-zA-Z ]', '')
df['country'] = df.country.str.replace('[^a-zA-Z ]', '')
df['language'] = df.language.str.replace('[^a-zA-Z ]', '')
df['director'] = df.director.str.replace('[^a-zA-Z ]', '')
df['writer'] = df.writer.str.replace('[^a-zA-Z ]', '')
df['actors'] = df.actors.str.replace('[^a-zA-Z ]', '')
df['actors2'] = df.actors2.str.replace('[^a-zA-Z ]', '')
df['actors3'] = df.actors3.str.replace('[^a-zA-Z ]', '')

In [63]:
label = df['avg_vote'].astype(int)
x = df.drop(['avg_vote'], axis=1)

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x, label, test_size=0.2, random_state=42)

In [ ]:
#df['year'] = pd.to_numeric(df['year'], errors='coerce')
#df = df[df['year'] > 2000]

In [65]:
x_train

,genre,genre2,genre3,duration,country,language,director,writer,actors,actors2,actors3
17490,Comedy,Western,None,102,USA,English,Don Taylor,Richard Alan Shapiro,Lee Marvin,Oliver Reed,Robert Culp
72858,Drama,Romance,None,155,India,Hindi,Vivek Agnihotri,Vivek Agnihotri,Pulkit Samrat,Yami Gautam,Gulshan Devaiah
8627,Comedy,Drama,None,101,Mexico,Spanish,Miguel M Delgado,Daniel Jimnez,Cantinflas,Manola Saavedra,Flor Silvestre
71878,Horror,Thriller,None,70,USA,English,Albert Pyun,Cynthia Curnan,Shane Ryan,Tommie Vegas,Brittany Bochart
59725,Comedy,Drama,None,83,USA,English,Jay Duplass,Jay Duplass,Jason Segel,Ed Helms,Susan Sarandon
...,...,...,...,...,...,...,...,...,...,...,...
6487,Drama,Mystery,None,89,Sweden,Swedish,Hasse Ekman,Hasse Ekman,Eva Henning,Ulf Palme,Birgit Tengroth
56064,Romance,None,None,106,Taiwan,Mandarin,Hungi Chen,Hungi Chen,Josephine A Blankstein,Cheer Chen,Yangkun Chen
78829,Drama,Fantasy,Horror,103,USA,English,Assaf Bernstein,Assaf Bernstein,India Eisley,Jason Isaacs,Mira Sorvino
1052,Comedy,Fantasy,Romance,95,USA,English,David Butler,William M Conselman,Will Rogers,William Farnum,Frank Albertson


In [66]:
le_genre = LabelEncoder()
ce_lan = ce.BaseNEncoder(cols=['language'],base=3,return_df=True)
ce_country = ce.BaseNEncoder(cols=['country'],base=3,return_df=True)
ce_director = ce.BaseNEncoder(cols=['director'],base=10,return_df=True)
ce_writer = ce.BaseNEncoder(cols=['writer'],base=10,return_df=True)
ce_actors = ce.BaseNEncoder(cols=['actors','actors2','actors3'],base=10,return_df=True)

In [67]:
le_genre.fit(pd.concat([df['genre'], df['genre2'], df['genre3']]).unique())
x_train['genre'] = le_genre.transform(x_train['genre'])
x_train['genre2'] = le_genre.transform(x_train['genre2'])
x_train['genre3'] = le_genre.transform(x_train['genre3'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [68]:
x_train = ce_lan.fit_transform(x_train)
x_train = ce_country.fit_transform(x_train)
x_train = ce_director.fit_transform(x_train)
x_train = ce_writer.fit_transform(x_train)
x_train = ce_actors.fit_transform(x_train)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: Future

In [69]:
mms = MinMaxScaler()
ct = ColumnTransformer([
        ('duration_scl', mms, ['duration'])
    ])
temp_array = ct.fit(x_train)
temp_array = ct.transform(x_train)
x_train = x_train.reset_index()
x_train = x_train.drop(columns=['index'])
x_train["duration"] = pd.DataFrame(temp_array, columns = ['duration'])

In [70]:
x_train

,genre,genre2,genre3,duration,country_0,country_1,country_2,country_3,country_4,country_5,language_0,language_1,language_2,language_3,language_4,language_5,director_0,director_1,director_2,director_3,director_4,director_5,writer_0,writer_1,writer_2,writer_3,writer_4,writer_5,actors_0,actors_1,actors_2,actors_3,actors_4,actors_5,actors2_0,actors2_1,actors2_2,actors2_3,actors2_4,actors2_5,actors3_0,actors3_1,actors3_2,actors3_3,actors3_4,actors3_5
0,5,25,18,0.079531,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,8,20,18,0.148631,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,2
2,5,8,18,0.078227,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,0,0,0,3,0,0,0,0,0,3,0,0,0,0,0,3,0,0,0,0,0,3
3,13,23,18,0.037810,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,4,0,0,0,0,0,4,0,0,0,0,0,4,0,0,0,0,0,4,0,0,0,0,0,4
4,5,8,18,0.054759,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66705,8,16,18,0.062581,0,0,0,2,2,0,0,0,0,2,2,0,0,0,6,3,3,7,0,0,1,8,2,6,0,3,0,9,8,2,0,2,4,1,5,1,0,4,2,1,9,8
66706,20,18,18,0.084746,0,0,0,1,0,1,0,0,0,1,0,1,0,2,7,4,8,0,0,3,7,1,4,8,0,3,0,9,8,3,0,3,7,7,2,6,0,4,2,1,9,9
66707,8,10,13,0.080834,0,0,0,0,0,1,0,0,0,0,0,1,0,2,7,4,8,1,0,3,7,1,4,9,0,0,3,1,6,5,0,0,5,9,4,6,0,0,9,7,8,5
66708,5,10,20,0.070404,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,7,5,8,0,1,9,0,4,1,0,0,5,1,2,8,0,3,7,7,2,7,0,0,5,5,2,5


In [71]:
x_test

,genre,genre2,genre3,duration,country,language,director,writer,actors,actors2,actors3
44934,Comedy,Drama,None,102,Netherlands,Dutch,Eddy Terstall,Eddy Terstall,Cees Geel,Marcel Hensema,Rifka Lodeizen
57358,Horror,Thriller,None,89,Germany,German,Robert Franke,Raimund V Tabor,Diana Radnai,Vera Hbner,Thomas Harbort
16948,Drama,None,None,91,West,Persian,Sohrab Shahid Saless,Helga Houzer,Parviz Sayyad,Cihan Anasai,Muhammet Temizkan
36007,Crime,Drama,None,102,Turkey,Turkish,Serdar Akar,Serdar Akar,Erkan Can,Haldun Boysan,Yildiray Sahinler
49289,Comedy,Romance,None,89,USA,English,Rolfe Kanefsky,Rolfe Kanefsky,Eric Fagundes,Julia Lehman,Valerie Carpenter
...,...,...,...,...,...,...,...,...,...,...,...
76068,Horror,None,None,97,USA,English,Bryan Brewer,Bryan Brewer,Bryan Brewer,Cree Kelly,Allie Rivera
23163,Action,Crime,Drama,89,USA,English,Peter Manoogian,Stuart Kaminsky,Gary Frank,Ray Parker Jr,JanMichael Vincent
11124,Drama,None,None,93,Italy,Italian,Damiano Damiani,Elsa Morante,Vanni De Maigret,Key Meersman,Reginald Kernan
40415,Action,Crime,Drama,141,India,Hindi,Mahesh Bhatt,Robin Bhatt,Akshay Kumar,Nagarjuna Akkineni,Pooja Bhatt


In [72]:
x_test['genre'] = le_genre.transform(x_test['genre'])
x_test['genre2'] = le_genre.transform(x_test['genre2'])
x_test['genre3'] = le_genre.transform(x_test['genre3'])
x_test = ce_lan.transform(x_test)
x_test = ce_country.transform(x_test)
x_test = ce_director.transform(x_test)
x_test = ce_writer.transform(x_test)
x_test = ce_actors.transform(x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [73]:
temp_array = ct.transform(x_test)
x_test = x_test.reset_index()
x_test = x_test.drop(columns=['index'])
x_test["duration"] = pd.DataFrame(temp_array, columns = ['duration'])

In [74]:
x_test

,genre,genre2,genre3,duration,country_0,country_1,country_2,country_3,country_4,country_5,language_0,language_1,language_2,language_3,language_4,language_5,director_0,director_1,director_2,director_3,director_4,director_5,writer_0,writer_1,writer_2,writer_3,writer_4,writer_5,actors_0,actors_1,actors_2,actors_3,actors_4,actors_5,actors2_0,actors2_1,actors2_2,actors2_3,actors2_4,actors2_5,actors3_0,actors3_1,actors3_2,actors3_3,actors3_4,actors3_5
0,5,8,18,0.079531,0,0,0,1,1,2,0,0,0,1,1,1,0,0,4,7,0,7,0,0,7,2,8,0,0,0,0,0,0,0,0,2,2,3,9,9,0,2,8,9,2,7
1,13,23,18,0.062581,0,0,1,0,0,1,0,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,18,18,0.065189,0,0,1,1,0,2,0,0,1,1,1,0,0,0,5,0,0,1,0,0,0,0,0,0,0,0,5,1,8,3,0,0,0,0,0,0,0,0,0,0,0,0
3,6,8,18,0.079531,0,0,0,0,2,0,0,0,0,0,1,2,0,0,1,1,8,1,0,1,0,4,0,0,0,1,3,2,9,6,0,2,3,7,8,8,0,0,0,0,0,0
4,5,20,18,0.062581,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,6,7,9,0,0,5,4,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16673,13,18,18,0.073012,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,9,6,0,2,8,7,9,8,0,3,2,1,8,0,0,0,0,0,0,0
16674,0,6,8,0.062581,0,0,0,0,0,1,0,0,0,0,0,1,0,0,7,9,8,7,0,3,6,3,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,2,9,4
16675,8,18,18,0.067797,0,0,0,1,2,1,0,0,0,2,0,2,0,0,4,9,3,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16676,0,6,8,0.130378,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,2,6,4,0,0,0,4,7,2,0,0,0,1,1,2,0,0,6,7,3,5,0,0,1,0,2,5


In [75]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train,y_train)
predictions = model.predict(x_test)
print( accuracy_score(y_test, predictions))

0.39561098453051924


In [76]:
dump(model, open('random_forest_movie_rating_model.pkl', 'wb'))

In [77]:
dump(ct, open('duration_scaler.pkl', 'wb'))

dump(le_genre, open('encoder_genre.pkl', 'wb'))

dump(ce_lan, open('encoder_lan.pkl', 'wb'))

dump(ce_country, open('encoder_country.pkl', 'wb'))

dump(ce_director, open('encoder_director.pkl', 'wb'))

dump(ce_writer, open('encoder_writer.pkl', 'wb'))

dump(ce_actors, open('encoder_actors.pkl', 'wb'))